In [1]:
import folium

print(folium.__file__)
print(folium.__version__)

/home/filipe/.virtualenvs/iris/lib/python2.7/site-packages/folium/__init__.py
0.1.6


In [2]:
import numpy as np
import pandas as pd
import numpy.ma as ma


def make_data():
    x = np.linspace(-np.pi, np.pi, 101)
    sin = np.sin(x)
    cos = np.cos(x)
    cos[20:50] = np.NaN
    return pd.DataFrame(np.asanyarray([sin, cos]).T, columns=['sin', 'cos'], index=x)

In [3]:
station = '42'
lon, lat = -42, -21
mapa = folium.Map(location=[lat, lon], zoom_start=5)

df = make_data()
resolution, width, height = 75, 7, 3

### Simple text

In [4]:
popup = "I should be texting a big chunk of text but I am lazy...\n" * 10
kw = dict(popup=popup, marker_color="black", marker_icon="ok")
mapa.simple_marker(location=[lat-5, lon-5], **kw)

### PNG

In [5]:
import base64
import matplotlib.pyplot as plt


fig, ax = plt.subplots(figsize=(width, height))
ax = df.plot(ax=ax, legend=False)
ax.set_ylabel('Sea surface height (m)')
png = 'mpld3_{}.png'.format(station)
fig.savefig(png, dpi=resolution)

popup = """<div align='center'> {} <br><img src='{}' width='{}' height='{}'> </div>"""
popup = popup.format('PNG', png, width*resolution, height*resolution)
kw = dict(popup=popup, popup_width=width*resolution, marker_color="red", marker_icon="ok")

mapa.simple_marker(location=[lat-2, lon-1], **kw)

### Vega/Vincent HTML (does not work with missing values)

In [6]:
import vincent


df.fillna(value='null', inplace=True)
vis = vincent.Line(df, width=width*resolution, height=height*resolution)
vis.legend(title='Vega')
json = 'vega_{}.json'.format(station)
vis.to_json(json)

popup = (vis, json)
kw.update(dict(marker_color="green", popup=popup))

mapa.simple_marker(location=[lat-1, lon+1], **kw)

### mpld3 HTML

In [7]:
import mpld3
from mpld3.plugins import LineLabelTooltip, connect


fig, ax = plt.subplots(figsize=(width, height))
ax = df.plot(ax=ax, legend=False)
ax.set_ylabel('Sea surface height (m)')

[mpld3.plugins.connect(fig, LineLabelTooltip(line, name)) for
 line, name in zip(ax.lines, df.columns)]

html = 'mpld3_{}.html'.format(station)
mpld3.save_html(fig, html)

iframe = ("<div align='center'> {} <br><iframe src='{}' "
          "alt='mpld3 image' width='{}px' height='{}px' frameBorder='0'></div>")

popup = iframe.format('HTML', html, (width*resolution)+75, (height*resolution)+50)
kw.update(dict(popup=popup, marker_color="blue", popup_width=(width*resolution)+75))

mapa.simple_marker(location=[lat+1, lon-1], **kw)

### Just an idea: mpld3 JSON (akin to Vega)

In [8]:
if False:
    json = 'mpld3_{}.json'.format(station)
    mpld3.save_json(fig, json)

In [9]:
mapa # The new _repr_html is broken here. Need to carefully encode all pieces before hand.

In [10]:
mapa.create_map("mapa.html")